# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [30]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,-0.05,99,100,1.94,SJ,1702347208
1,1,puerto ayora,-0.7393,-90.3518,24.17,86,10,2.85,EC,1702347208
2,2,leer,53.2316,7.4610,7.19,92,90,3.48,DE,1702347208
3,3,saint-joseph,-21.3667,55.6167,23.21,76,41,3.70,RE,1702347208
4,4,ballina,-28.8667,153.5667,28.77,57,65,3.54,AU,1702347208


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
#%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# My Criteria:
# max temperature lower than 27 degrees but higher than 21
# cloudiness less than 2
# wind speed less than 4.5 m/s

# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 2)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,margaret river,-33.9500,115.0667,25.43,46,0,2.65,AU,1702347217
76,76,alice springs,-23.7000,133.8833,25.75,29,0,2.57,AU,1702347219
101,101,ziniare,12.5822,-1.2983,23.02,22,0,4.33,BF,1702347222
217,217,cabo san lucas,22.8909,-109.9124,23.75,75,0,0.00,MX,1702347197
280,280,maragogi,-9.0122,-35.2225,25.48,76,0,3.87,BR,1702347238
282,282,vila velha,-20.3297,-40.2925,26.08,84,0,3.58,BR,1702347154
291,291,tecuala,22.3986,-105.4569,22.45,74,0,0.42,MX,1702347163
416,416,tamandare,-8.7597,-35.1047,25.35,77,0,3.45,BR,1702347190
417,417,maraba,-5.3686,-49.1178,25.10,88,0,1.03,BR,1702347253
461,461,porto novo,6.4965,2.6036,26.90,85,0,0.53,BJ,1702347258


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
49,margaret river,AU,-33.9500,115.0667,46,
76,alice springs,AU,-23.7000,133.8833,29,
101,ziniare,BF,12.5822,-1.2983,22,
217,cabo san lucas,MX,22.8909,-109.9124,75,
280,maragogi,BR,-9.0122,-35.2225,76,
282,vila velha,BR,-20.3297,-40.2925,84,
291,tecuala,MX,22.3986,-105.4569,74,
416,tamandare,BR,-8.7597,-35.1047,77,
417,maraba,BR,-5.3686,-49.1178,88,
461,porto novo,BJ,6.4965,2.6036,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 5000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
margaret river - nearest hotel: Margaret River Hotel
alice springs - nearest hotel: Aurora Alice Springs
ziniare - nearest hotel: Residence Les Saints Innocents
cabo san lucas - nearest hotel: Comfort Rooms
maragogi - nearest hotel: Pousada Mariluz
vila velha - nearest hotel: Hotel Prainha
tecuala - nearest hotel: No hotel found
tamandare - nearest hotel: Pousada Baia dos Corais
maraba - nearest hotel: Portobelo
porto novo - nearest hotel: Hôtel 6500 CFA


,City,Country,Lat,Lng,Humidity,Hotel Name
49,margaret river,AU,-33.9500,115.0667,46,Margaret River Hotel
76,alice springs,AU,-23.7000,133.8833,29,Aurora Alice Springs
101,ziniare,BF,12.5822,-1.2983,22,Residence Les Saints Innocents
217,cabo san lucas,MX,22.8909,-109.9124,75,Comfort Rooms
280,maragogi,BR,-9.0122,-35.2225,76,Pousada Mariluz
282,vila velha,BR,-20.3297,-40.2925,84,Hotel Prainha
291,tecuala,MX,22.3986,-105.4569,74,No hotel found
416,tamandare,BR,-8.7597,-35.1047,77,Pousada Baia dos Corais
417,maraba,BR,-5.3686,-49.1178,88,Portobelo
461,porto novo,BJ,6.4965,2.6036,85,Hôtel 6500 CFA


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
#%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)